In [3]:
from fastapi import FastAPI, HTTPException
import requests
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# Inisialisasi FastAPI
app = FastAPI()

# Masukkan API Key OpenWeatherMap Anda
API_KEY = "1126e6877285c03932db1caea51882a5" 
BASE_URL_CURRENT = "https://api.openweathermap.org/data/2.5/weather"
BASE_URL_FORECAST = "https://api.openweathermap.org/data/2.5/forecast"

@app.get("/weather/")
def get_current_weather(city: str):
    """
    Mengambil data cuaca saat ini berdasarkan nama kota.
    """
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }

    response = requests.get(BASE_URL_CURRENT, params=params)
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Error fetching current weather data.")

    data = response.json()

    return {
        "city": data["name"],
        "temperature": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "description": data["weather"][0]["description"],
        "wind_speed": data["wind"]["speed"]
    }

@app.get("/forecast/")
def get_weather_forecast(city: str):
    """
    Mengambil data prediksi cuaca 5 hari ke depan berdasarkan nama kota.
    """
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"
    }

    response = requests.get(BASE_URL_FORECAST, params=params)
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Error fetching forecast data.")

    data = response.json()

    # Menyusun data menjadi daftar prediksi yang disederhanakan
    forecast = []
    for item in data["list"]:
        forecast.append({
            "datetime": item["dt_txt"],
            "temperature": item["main"]["temp"],
            "humidity": item["main"]["humidity"],
            "description": item["weather"][0]["description"],
            "wind_speed": item["wind"]["speed"]
        })

    return {
        "city": data["city"]["name"],
        "country": data["city"]["country"],
        "forecast": forecast
    }

# Menjalankan server
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8600)

# Jalankan aplikasi
run_server()

INFO:     Started server process [35748]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8600 (Press CTRL+C to quit)


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [35748]
